In [1]:
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit"

import tensorflow as tf
tf.enable_eager_execution()
# tf.enable_v2_behavior()
# sess = tf.InteractiveSession()
# sess.as_default()

import util_funcs
from importlib import reload
import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import clinical_text_analysis as cta
import tsfresh
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from os import path
import predictSeizureConvExp as psce
import keras_models.dataGen as dg
from addict import Dict
reload(psce)
from keras.utils import multi_gpu_model
import keras.optimizers
import ensembleReader as er
import time
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, log_loss
from functools import lru_cache

Using TensorFlow backend.


In [2]:
import tensorflow as tf
#https://androidkt.com/feed-tfrecord-to-keras/
def read_tfrecord(example):
    features = {'original_index': tf.io.FixedLenFeature([1], tf.int64, ),\
               'data':  tf.FixedLenFeature([9*21*1000], tf.float32,),\
               'label':  tf.FixedLenFeature([10], tf.int64, [0 for i in range(10)]),\
               'subtypeLabel':  tf.FixedLenFeature([10], tf.int64, [0 for i in range(10)]),\
               'patient':  tf.FixedLenFeature([1], tf.int64,), \
               'session':  tf.FixedLenFeature([1], tf.int64,),
                       }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)
#     return example
    data = tf.reshape(example['data'], [9,21,1000,1])
#     data = (example['data'])
    class_label = tf.cast(example['label'], tf.int32)
    
    paddings = tf.constant([[0,0],[0,0,], [42, 42], [0,0]])
    # 'constant_values' is 0.
    # rank of 't' is 2.
    data_decoder = tf.pad(data, paddings, "CONSTANT")
    
    return data, (tf.one_hot(class_label, 2)[0:9], data_decoder)
def get_batched_dataset(filenames):
    BATCH_SIZE=32
    
    option_no_order = tf.data.Options()
    option_no_order.experimental_deterministic = False

    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.with_options(option_no_order)
    dataset = dataset.interleave(tf.data.TFRecordDataset, cycle_length=16, num_parallel_calls=4)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=4)

#     dataset = dataset.cache() # This dataset fits in RAM
    dataset = dataset.repeat()
    dataset = dataset.shuffle(256)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) 
    dataset = dataset.prefetch(20) #

    return dataset
# featureData = read_tfrecord(allData[1])
import tensorflow as tf
def get_training_dataset():
    training_filenames = ["/n/scratch2/ms994/medium_size/train_1.tfrecords","/n/scratch2/ms994/medium_size/train_2.tfrecords","/n/scratch2/ms994/medium_size/train_3.tfrecords","/n/scratch2/ms994/medium_size/train_0.tfrecords"]
    return get_batched_dataset(training_filenames)

def get_validation_dataset():
    validation_filenames=["/n/scratch2/ms994/medium_size/valid_1.tfrecords","/n/scratch2/ms994/medium_size/valid_2.tfrecords","/n/scratch2/ms994/medium_size/valid_3.tfrecords","/n/scratch2/ms994/medium_size/valid_0.tfrecords"]
    return get_batched_dataset(validation_filenames)

def get_test_dataset():
    test_filenames=["/n/scratch2/ms994/medium_size/test_1.tfrecords","/n/scratch2/ms994/medium_size/test_2.tfrecords","/n/scratch2/ms994/medium_size/test_3.tfrecords","/n/scratch2/ms994/medium_size/test_0.tfrecords"]
    return get_batched_dataset(test_filenames)
negative_train_filenames =  ["/n/scratch2/ms994/medium_size/train_neg_1.tfrecords","/n/scratch2/ms994/medium_size/train_neg_2.tfrecords","/n/scratch2/ms994/medium_size/train_neg_3.tfrecords","/n/scratch2/ms994/medium_size/train_neg_0.tfrecords"]
positive_train_filenames =  ["/n/scratch2/ms994/medium_size/train_pos_1.tfrecords","/n/scratch2/ms994/medium_size/train_pos_2.tfrecords","/n/scratch2/ms994/medium_size/train_pos_3.tfrecords","/n/scratch2/ms994/medium_size/train_pos_0.tfrecords"]
    
def get_positive_train_dataset():
    return get_batched_dataset(positive_train_filenames)

def get_negative_train_dataset():
    return get_batched_dataset(negative_train_filenames)

def get_balanced_train_dataset(batch_size=64, max_queue_size=10):
    pos_ds = get_positive_train_dataset()
    neg_ds = get_negative_train_dataset()
    resampled_ds = tf.data.experimental.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])

    return resampled_ds


In [3]:
train = get_balanced_train_dataset()
valid = get_validation_dataset()
test = get_test_dataset()

In [4]:
from keras_models.metrics import f1, sensitivity, specificity
inputX = tf.keras.layers.Input((9,21,1000,1))
x = inputX
# inputX = tf.keras.Input((21*1000*1,))
# x = tf.keras.GaussianNoise(4)(inputX)
# x = tf.keras.layers.Reshape((9,21,1000,1))(inputX)
#encoder
for i in range(6):
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)
    if i > 3:
        num_filters=4
    else:
        num_filters=8
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(num_filters, (3,3), activation="relu"))(x)
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool2D((1,2)))(x)
decoder_reshape_shape =  tf.keras.backend.int_shape(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(x)
decoder_lstm_out_shape = tf.keras.backend.int_shape(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)
x = tf.keras.layers.CuDNNLSTM(128, return_sequences=True)(x)
x = tf.keras.layers.Activation("relu")(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)
x = tf.keras.layers.CuDNNLSTM(64, return_sequences=True)(x)
x = tf.keras.layers.Activation("relu")(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(128, activation="relu"))(x)
encoder_out = x
#seizure detection
x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(0.5))(x)
y = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(2, activation="relu"), name="seizure_detection")(x)

#decoder
x = tf.keras.layers.CuDNNLSTM(decoder_lstm_out_shape[2], return_sequences=True)(encoder_out)
x = tf.keras.layers.Activation("relu")(x)
x = tf.keras.layers.Reshape(decoder_reshape_shape[1:])(x)
for i in range(6):
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization())(x)
    if i > 3:
        num_filters=8
    else:
        num_filters=4
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2DTranspose(num_filters, (3,3), activation="relu"))(x)
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.UpSampling2D((1,2)))(x)

y_decoder = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2DTranspose(1, (1,1), activation="relu"), name="upsample")(x)
# y_decoder.name = x
# y_decoder = tf.keras.layers.Reshape((9,21,1000,1), name="decoder")(x)

model = tf.keras.Model(inputs=[inputX], outputs=[y, y_decoder])
model.compile(tf.keras.optimizers.Adam(lr=0.0001), loss={"seizure_detection":"categorical_crossentropy","upsample":"mse"}, metrics={"seizure_detection":["binary_accuracy", f1, sensitivity, specificity],"upsample":[]}, )

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


In [5]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 9, 21, 1000,  0                                            
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, 9, 21, 1000,  4           input_1[0][0]                    
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 9, 19, 998, 8 80          time_distributed[0][0]           
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 9, 19, 499, 8 0           time_distributed_1[0][0]         
__________________________________________________________________________________________________
time_distr

In [6]:
cb = [tf.keras.callbacks.EarlyStopping(monitor="val_seizure_detection_f1", mode="max", patience=10, verbose=True), ] #tf.keras.callbacks.ModelCheckpoint("/home/ms994/lstm_1.h5", monitor="val_seizure_detection_f1", mode="max", save_best_only=True, verbose=True)

In [7]:
history = model.fit(train, steps_per_epoch=200, validation_data=valid,  validation_steps=100, epochs=30, callbacks=cb)

Epoch 1/30
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


200/200 [==============================] - 133s 666ms/step - loss: 4.4444 - seizure_detection_loss: 4.4161 - upsample_loss: 0.0283 - seizure_detection_binary_accuracy: 0.4947 - seizure_detection_f1: 0.4038 - seizure_detection_sensitivity: 0.3537 - seizure_detection_specificity: 0.5332 - val_loss: 0.9797 - val_seizure_detection_loss: 0.9797 - val_upsample_loss: 3.1196e-05 - val_seizure_detection_binary_accuracy: 0.4475 - val_seizure_detection_f1: 0.0502 - val_seizure_detection_sensitivity: 0.4500 - val_seizure_detection_specificity: 0.0000e+00
Epoch 2/30
200/200 [==============================] - 121s 603ms/step - loss: 4.2180 - seizure_detection_loss: 4.2174 - upsample_loss: 5.8352e-04 - seizure_detection_binary_accuracy: 0.4708 - seizure_detection_f1: 0.3177 - seizure_detection_sensitivity: 0.2765 - seizure_detection_specificity: 0.6368 - val_loss: 0.7736 - val_seizure_detection_loss: 0.7734 - val_upsample_loss: 1.3507e-04 - val_seizure_detection_binary_accuracy: 0.4479 - val_seizure_

Epoch 11/30
200/200 [==============================] - 120s 600ms/step - loss: 1.0366 - seizure_detection_loss: 1.0366 - upsample_loss: 4.3176e-06 - seizure_detection_binary_accuracy: 0.4981 - seizure_detection_f1: 0.3340 - seizure_detection_sensitivity: 0.2616 - seizure_detection_specificity: 0.5489 - val_loss: 0.5811 - val_seizure_detection_loss: 0.5811 - val_upsample_loss: 2.8055e-06 - val_seizure_detection_binary_accuracy: 0.5000 - val_seizure_detection_f1: 0.1425 - val_seizure_detection_sensitivity: 0.1812 - val_seizure_detection_specificity: 0.9845
Epoch 12/30
200/200 [==============================] - 121s 605ms/step - loss: 0.9599 - seizure_detection_loss: 0.9599 - upsample_loss: 3.4015e-06 - seizure_detection_binary_accuracy: 0.5004 - seizure_detection_f1: 0.3069 - seizure_detection_sensitivity: 0.2385 - seizure_detection_specificity: 0.5853 - val_loss: 0.5693 - val_seizure_detection_loss: 0.5693 - val_upsample_loss: 2.2759e-06 - val_seizure_detection_binary_accuracy: 0.5000 -

Epoch 21/30
200/200 [==============================] - 121s 603ms/step - loss: 0.7689 - seizure_detection_loss: 0.7689 - upsample_loss: 6.0673e-07 - seizure_detection_binary_accuracy: 0.5007 - seizure_detection_f1: 0.2223 - seizure_detection_sensitivity: 0.1594 - seizure_detection_specificity: 0.6490 - val_loss: 0.5452 - val_seizure_detection_loss: 0.5452 - val_upsample_loss: 2.9888e-07 - val_seizure_detection_binary_accuracy: 0.5000 - val_seizure_detection_f1: 0.1052 - val_seizure_detection_sensitivity: 0.1018 - val_seizure_detection_specificity: 0.9776
Epoch 22/30
200/200 [==============================] - 120s 602ms/step - loss: 0.7667 - seizure_detection_loss: 0.7667 - upsample_loss: 5.2962e-07 - seizure_detection_binary_accuracy: 0.5002 - seizure_detection_f1: 0.2258 - seizure_detection_sensitivity: 0.1593 - seizure_detection_specificity: 0.6246 - val_loss: 0.5381 - val_seizure_detection_loss: 0.5381 - val_upsample_loss: 2.3666e-07 - val_seizure_detection_binary_accuracy: 0.5000 -

In [66]:
decoder_lstm_out_shape, decoder_reshape_shape

((None, 9, 3744), (None, 9, 9, 13, 32))

In [59]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 9, 21, 1000, 1)    0         
_________________________________________________________________
time_distributed_400 (TimeDi (None, 9, 21, 1000, 1)    4         
_________________________________________________________________
time_distributed_401 (TimeDi (None, 9, 19, 998, 1)     10        
_________________________________________________________________
time_distributed_402 (TimeDi (None, 9, 19, 499, 1)     0         
_________________________________________________________________
time_distributed_403 (TimeDi (None, 9, 19, 499, 1)     4         
_________________________________________________________________
time_distributed_404 (TimeDi (None, 9, 17, 497, 2)     20        
_________________________________________________________________
time_distributed_405 (TimeDi (None, 9, 17, 248, 2)     0         
__________

In [56]:
model.fit(train, steps_per_epoch=100)

100/100 [==============================] - 23s 226ms/step - loss: 2.7102 - binary_accuracy: 0.6417
